In [3]:
import pandas as pd
import sqlite3


In [5]:
orders_df = pd.read_csv("orders.csv")
orders_df.head()
orders_df.columns

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name'],
      dtype='object')

In [7]:
users_df = pd.read_json("users.json")
users_df.head()
users_df.columns

Index(['user_id', 'name', 'city', 'membership'], dtype='object')

In [9]:
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()
with open("restaurants.sql", "r", encoding="utf-8") as f:
    sql_script = f.read()

cursor.executescript(sql_script)

restaurants_df = pd.read_sql_query(
    "SELECT * FROM restaurants", conn
)


In [11]:
restaurants_df.head()
restaurants_df.columns


Index(['restaurant_id', 'restaurant_name', 'cuisine', 'rating'], dtype='object')

In [13]:
merged_df = pd.merge(
    orders_df,
    users_df,
    on="user_id",
    how="left"
)


In [15]:
final_df = pd.merge(
    merged_df,
    restaurants_df,
    on="restaurant_id",
    how="left"
)


In [18]:
final_df.to_csv(
    "final_food_delivery_dataset.csv",
    index=False
)

In [20]:
final_df.shape
final_df.isnull().sum()


order_id             0
user_id              0
restaurant_id        0
order_date           0
total_amount         0
restaurant_name_x    0
name                 0
city                 0
membership           0
restaurant_name_y    0
cuisine              0
rating               0
dtype: int64

In [22]:
import os
print(os.getcwd())



C:\Users\saram


In [24]:
final=pd.read_csv("final_food_delivery_dataset.csv")
final.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [32]:
final['order_date'] = pd.to_datetime(final['order_date'])
final['total_amount'] = pd.to_numeric(final['total_amount'], errors='coerce')
final['rating'] = pd.to_numeric(final['rating'], errors='coerce')


In [40]:
###Which city has the highest total revenue (total_amount) from Gold members?  
gold_city_revenue = (
    final[final['membership'] == 'Gold']
    .groupby('city')['total_amount']
    .sum()
    .sort_values(ascending=False)
)

gold_city_revenue.head()


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [44]:
###Cuisine with Highest Average Order Value
cuisine_aov = (
    final.groupby('cuisine')['total_amount']
    .mean()
    .sort_values(ascending=False)
)

cuisine_aov


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [48]:
###Distinct Users with Total Spend > ₹1000
user_total_spend = (
    final.groupby('user_id')['total_amount']
    .sum()
)

count_users = (user_total_spend > 1000).sum()
count_users


2544

In [50]:
count_users


2544

In [56]:
###Rating Range Generating Highest Total Revenue
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0–3.5', '3.6–4.0', '4.1–4.5', '4.6–5.0']

final['rating_range'] = pd.cut(final['rating'], bins=bins, labels=labels, include_lowest=True)

rating_revenue = (
    final.groupby('rating_range')['total_amount']
    .sum()
    .sort_values(ascending=False)
)

rating_revenue


C:\Users\saram\AppData\Local\Temp\ipykernel_14092\3884573547.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final.groupby('rating_range')['total_amount']


rating_range
4.6–5.0    2197030.75
3.0–3.5    2136772.70
4.1–4.5    1960326.26
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

In [60]:
###Gold Members: City with Highest Average Order Value
gold_city_aov = (
    final[final['membership'] == 'Gold']
    .groupby('city')['total_amount']
    .mean()
    .sort_values(ascending=False)
)

gold_city_aov


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [66]:
cuisine_stats = (
    final.groupby('cuisine')
    .agg(
        distinct_restaurants=('restaurant_id', 'nunique'),
        total_revenue=('total_amount', 'sum')
    )
    .sort_values(
        ['distinct_restaurants', 'total_revenue'],
        ascending=[True, False]
    )
)

cuisine_stats


,distinct_restaurants,total_revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


In [70]:
###Percentage of total orders placed by Gold members
total_orders = len(final)
gold_orders = len(final[final['membership'] == 'Gold'])

percentage_gold = round((gold_orders / total_orders) * 100)
percentage_gold

50

In [72]:
###Restaurant with highest AOV but < 20 total orders
restaurant_stats = (
    final.groupby('restaurant_name_y')
    .agg(
        avg_order_value=('total_amount', 'mean'),
        total_orders=('order_id', 'count')
    )
)

filtered = restaurant_stats[restaurant_stats['total_orders'] < 20]

filtered.sort_values('avg_order_value', ascending=False)


,avg_order_value,total_orders
restaurant_name_y,,
Restaurant_294,1040.222308,13
Restaurant_262,1029.473333,18
Restaurant_77,1029.180833,12
Restaurant_193,1026.306667,15
Restaurant_7,1002.140625,16
...,...,...
Restaurant_184,621.828947,19
Restaurant_498,596.815556,18
Restaurant_192,589.972857,14


In [80]:
final[['restaurant_name_x', 'restaurant_name_y']]


,restaurant_name_x,restaurant_name_y
0,New Foods Chinese,Restaurant_450
1,Ruchi Curry House Multicuisine,Restaurant_309
2,Spice Kitchen Punjabi,Restaurant_107
3,Darbar Kitchen Non-Veg,Restaurant_224
4,Royal Eatery South Indian,Restaurant_293
...,...,...
9995,Royal Kitchen North Indian,Restaurant_249
9996,Darbar Cafe Punjabi,Restaurant_267
9997,Ruchi Tiffins Chinese,Restaurant_420
9998,Swagath Kitchen North Indian,Restaurant_492


,avg_order_value,total_orders
restaurant_name_y,,


In [78]:
final['restaurant_name_y'].isin([
    'Grand Cafe Punjabi',
    'Grand Restaurant South Indian',
    'Ruchi Mess Multicuisine',
    'Ruchi Foods Chinese'
]).any()


False

In [82]:
###Which combination contributes the highest revenue?
combo_revenue = (
    final.groupby(['membership', 'cuisine'])['total_amount']
    .sum()
    .sort_values(ascending=False)
)

combo_revenue

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [84]:
###During which quarter of the year is the total revenue highest?
final['order_date'] = pd.to_datetime(final['order_date'], format='%d-%m-%Y')
final['quarter'] = final['order_date'].dt.to_period('Q')
quarter_revenue = (
    final.groupby('quarter')['total_amount']
    .sum()
    .sort_values(ascending=False)
)

quarter_revenue


quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

In [86]:
###How many total orders were placed by users with Gold membership?
gold_total_orders = final[final['membership'] == 'Gold']['order_id'].nunique()
gold_total_orders

4987

In [88]:
###Total revenue (rounded to nearest integer) from Hyderabad city
hyderabad_revenue = final[final['city'] == 'Hyderabad']['total_amount'].sum()
round(hyderabad_revenue)

1889367

In [90]:
###How many distinct users placed at least one order?
distinct_users = final['user_id'].nunique()
distinct_users

2883

In [92]:
###Average order value (rounded to 2 decimals) for Gold members
gold_aov = final[final['membership'] == 'Gold']['total_amount'].mean()
round(gold_aov, 2)

797.15

In [94]:
###How many orders were placed for restaurants with rating ≥ 4.5?
high_rating_orders = final[final['rating'] >= 4.5]['order_id'].nunique()
high_rating_orders

3374

In [96]:
###How many orders were placed in the top revenue city among Gold members only?
top_gold_city = (
    final[final['membership'] == 'Gold']
    .groupby('city')['total_amount']
    .sum()
    .idxmax()
)

top_gold_city


'Chennai'

In [98]:
orders_in_top_gold_city = final[
    (final['membership'] == 'Gold') &
    (final['city'] == top_gold_city)
]['order_id'].nunique()

orders_in_top_gold_city


1337

In [100]:
len(final)


10000